In [20]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

In [2]:
print(tf.__version__)

1.7.0


In [3]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

### 각 변수의 초기화를 일일이 실행하지 않고 모든 변수를 초기화하고 싶으면

In [10]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


- Jupyter나 Python shell 에선 InteractiveSession도 추천!
    - Session과 다른 점은 InteractiveSession이 만들어질 때 자신을 기본 세션으로 지정하는 것
    - with 블록을 사용할 필요 없음(그러나 수동으로 세션을 종료)

In [8]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

42


In [9]:
sess.close()

### Tensorflow
- 구성 단계
- 실행 단계

In [11]:
x1 = tf.Variable(1)

In [12]:
x1.graph is tf.get_default_graph()

True

In [13]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [14]:
x2.graph is graph

True

In [15]:
x2.graph is tf.get_default_graph()

False

In [16]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [17]:
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


- 위 코드는 w,x 를 두 번 평가함
- 한번에 모두 평가하도록 하려면 아래 코드처럼

In [18]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


# 회귀분석

In [21]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

In [23]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)), XT), y)

In [25]:
with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


In [26]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

In [27]:
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A: [[4,5,6], [7,8,9]]})

In [29]:
print(B_val_1)

[[6. 7. 8.]]


In [30]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


In [ ]:
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape(None, 1), name="y")

### Saver

In [ ]:
save = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            save_path = saver.save(sess, "./tmp/my_model.ckpt")
            
        sess.run(training_op)
        
    best_theta = theta.eval()
    save_path = saver.save(Sess, "./tmp/my_model_final.ckpt")

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./tmp/my_model_final.ckpt")

- saver를 세부적으로 제어하려면 저장 또는 복원할 변수를 지정하거나 이름을 사용

In [ ]:
saver = tf.train(saver({"weights": theta}))


- 그래프 구조와 변수값을 포함해 저장한 모델 복원하고 싶은 경우

In [ ]:
saver = tf.train.import_meta_graph("./tmp/my_model_final.ckpt.meta")

with tf.Session() as sess:
    saver.restore(sess, "./tmp/my_model_final.ckpt")
    
    

## 텐서보드

In [31]:
from datetime import datetime


In [34]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [36]:
mse_summary = tf.summary.scalar("MSE", mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [38]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")